# 🌳 Merkle Tree

Notebook com conceitos de:
- Merkle Tree (Árvore de Merkle)
- Merkle Proof (Prova de inclusão)

Utilizaremos apenas `Python` e a função `sha256` da biblioteca `hashlib`.


In [ ]:
import hashlib
import math

# Função de hash SHA-256
def sha256(data):
    if isinstance(data, str):
        data = data.encode('utf-8')
    return hashlib.sha256(data).hexdigest()


In [ ]:
class MerkleTree:
    def __init__(self, data_blocks):
        self.data_blocks = data_blocks
        self.levels = []
        self.build_tree()

    def build_tree(self):
        leaves = [sha256(block) for block in self.data_blocks]
        self.levels.append(leaves)
        current_level = leaves
        while len(current_level) > 1:
            next_level = []
            for i in range(0, len(current_level), 2):
                left = current_level[i]
                right = current_level[i+1] if i+1 < len(current_level) else left
                next_level.append(sha256(left + right))
            self.levels.append(next_level)
            current_level = next_level

    def root(self):
        return self.levels[-1][0] if self.levels else None

    def print_tree(self):
        for i, level in enumerate(self.levels):
            if i == len(self.levels) -1:
                print(f"Raiz :")
            else:
                print(f"Nível {i}:")
            for node in level:
                print(f"  {node}")


## 🌿 Exemplo: Construindo uma Merkle Tree simples

In [ ]:
data = ["bloco A", "bloco B", "bloco C", "bloco D"]
tree = MerkleTree(data)
tree.print_tree()


## 🔎 Merkle Proof: Verificando se um bloco pertence à árvore

In [ ]:
def get_merkle_proof(tree:MerkleTree, index):
    proof = []
    for level in tree.levels[:-1]:
        irmao = index ^ 1  # pega o vizinho (xor, bit flip)
        if irmao < len(level):
            proof.append(level[irmao])
        index //= 2
    return proof

def verify_merkle_proof(leaf_hash, proof, root_hash, index):
    current_hash = leaf_hash
    for sibling in proof:
        if index % 2 == 0:
            current_hash = sha256(current_hash + sibling)
        else:
            current_hash = sha256(sibling + current_hash)
        index //= 2
    return current_hash == root_hash


In [ ]:
leaf_index = 2  # "bloco C"
leaf_hash = sha256(data[leaf_index])
print("no [{}] : {}".format(data[leaf_index], leaf_hash))

In [ ]:
proof = get_merkle_proof(tree, leaf_index)
proof

In [ ]:
verify_merkle_proof(leaf_hash, proof, tree.root(), leaf_index)

um bloco não existente

In [ ]:
proof = get_merkle_proof(tree, leaf_index)
verify_merkle_proof(sha256("bloco Z"), proof, tree.root(), leaf_index)